![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20CamemBertForQuestionAnswering.ipynb)

## Import CamemBertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.2.7` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import CamemBERT models trained/fine-tuned for question answering via `CamembertForQuestionAnswering` or `TFCamembertForQuestionAnswering`. These models are usually under `Question Answering` category and have `camembert` in their labels
- Reference: [TFCamembertForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/camembert#transformers.TFCamembertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?other=camembert&pipeline_tag=question-answering&sort=downloads)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.25.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- CamemBERT uses SentencePiece, so we will have to install that as well


In [ ]:
!pip install -q transformers==4.25.1 tensorflow==2.11.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [etalab-ia/camembert-base-squadFR-fquad-piaf](https://huggingface.co/etalab-ia/camembert-base-squadFR-fquad-piaf) model from HuggingFace as an example
- In addition to `TFCamembertForQuestionAnswering` we also need to save the `CamembertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFCamembertForQuestionAnswering, CamembertTokenizer 
import tensorflow as tf

MODEL_NAME = 'illuin/camembert-base-fquad'

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFCamembertForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
  model = TFCamembertForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int64, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int64, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFCamembertForQuestionAnswering.

All the weights of TFCamembertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForQuestionAnswering for predictions without further training.


INFO:tensorflow:Assets written to: ./illuin/camembert-base-fquad/saved_model/1/assets


INFO:tensorflow:Assets written to: ./illuin/camembert-base-fquad/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 860208
-rw-r--r--  1 maziyar  staff        717 Dec 25 00:49 config.json
drwxr-xr-x  3 maziyar  staff         96 Dec 25 00:49 saved_model
-rw-r--r--  1 maziyar  staff  440418536 Dec 25 00:49 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 18808
drwxr-xr-x  2 maziyar  staff       64 Dec 25 00:49 assets
-rw-r--r--  1 maziyar  staff       54 Dec 25 00:49 fingerprint.pb
-rw-r--r--  1 maziyar  staff   165535 Dec 25 00:49 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  9454001 Dec 25 00:49 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 25 00:49 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 1600
-rw-r--r--  1 maziyar  staff  810912 Dec 25 00:48 sentencepiece.bpe.model
-rw-r--r--  1 maziyar  staff     241 Dec 25 00:48 special_tokens_map.json
-rw-r--r--  1 maziyar  staff     760 Dec 25 00:48 tokenizer_config.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `sentencepiece.bpe.model` from the tokenizer
- All we need is to just copy the `sentencepiece.bpe.model` to `saved_model/1/assets` which Spark NLP will look for

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

Voila! We have our `sentencepiece.bpe.model` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 1584
-rw-r--r--  1 maziyar  staff  810912 Dec 24 20:59 sentencepiece.bpe.model


## Import and Save CamemBertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 4.2.6
setup Colab for PySpark 3.2.3 and Spark NLP 4.2.6


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `CamemBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = CamemBertForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your CamemBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 2198320
drwxr-xr-x  3 maziyar  staff          96 Dec 15 18:30 fields
drwxr-xr-x  6 maziyar  staff         192 Dec 15 18:30 metadata
-rw-r--r--  1 maziyar  staff  1119269627 Dec 15 18:31 xlm_roberta_classification_tensorflow
-rw-r--r--  1 maziyar  staff     5069051 Dec 15 18:31 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForQuestionAnswering model in Spark NLP 🚀 pipeline! 

In [ ]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = CamemBertForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

context = """L'idée selon laquelle une planète inconnue pourrait exister entre les orbites de Mars et Jupiter fut proposée pour la première fois par Johann Elert Bode en 1768. Ses suggestions étaient
basées sur la loi de Titius-Bode, une théorie désormais obsolète proposée par Johann Daniel Titius en 1766,. Selon cette loi, le demi-grand axe de cette planète aurait été d'environ 2,8 ua. La découverte d'Uranus par William Herschel en
1781 accrut la confiance dans la loi de Titius-Bode et, en 1800, vingt-quatre astronomes expérimentés combinèrent leurs efforts et entreprirent une recherche méthodique de la planète proposée,. Le groupe était dirigé par
Franz Xaver von Zach. Bien qu'ils n'aient pas découvert Cérès, ils trouvèrent néanmoins plusieurs autres astéroïdes."""

question1 = "Quel astronome a émit l'idée en premier d'une planète entre les orbites de Mars et Jupiter ?"
question2 = "Quel astronome découvrit Uranus ?"
question3 = "Quelles furent les découvertes finales des vingt-quatre astronomes ?"

example = spark.createDataFrame([[question1, context], [question2, context], [question3, context]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("question", "answer.result").show(3, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `CamemBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀 
